In [6]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_chroma import Chroma
import numpy as np
from typing import List
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
model = init_chat_model("groq:qwen/qwen3-32b")


In [7]:
sample_docs = [
    """
    Retrieval-Augmented Generation (RAG) and Vector Databases
    
    RAG systems enhance Large Language Models by retrieving relevant data from external sources like ChromaDB or FAISS. 
    By converting documents into vector embeddings, the system can perform semantic searches to find context 
    that a model wasn't originally trained on, reducing hallucinations.
    """,
    
    """
    Agentic AI and Autonomous Workflows
    
    Agentic AI refers to systems designed to use tools and make decisions to achieve a goal. 
    Unlike standard chatbots, AI agents can use 'Reasoning and Acting' (ReAct) patterns to 
    call APIs, search the web, or execute code independently to complete multi-step tasks.
    """,
    
    """
    Cloud-Native Microservices and Scalability
    
    Modern backend architectures often utilize Spring Boot and Docker to create microservices. 
    Deploying these on AWS using services like EKS or Lambda allows for elastic scaling. 
    API Gateways act as the entry point, routing traffic to specific services like Product, Order, or User modules.
    """,
    
    """
    Natural Language Processing with NLTK and Transformers
    
    Natural Language Processing (NLP) involves the interaction between computers and human languages. 
    Libraries like NLTK are used for basic tokenization and stop-word removal, while Transformer-based 
    models like BERT or GPT handle complex tasks like sentiment analysis and language translation.
    """
]

In [8]:
sample_docs

["\n    Retrieval-Augmented Generation (RAG) and Vector Databases\n\n    RAG systems enhance Large Language Models by retrieving relevant data from external sources like ChromaDB or FAISS. \n    By converting documents into vector embeddings, the system can perform semantic searches to find context \n    that a model wasn't originally trained on, reducing hallucinations.\n    ",
 "\n    Agentic AI and Autonomous Workflows\n\n    Agentic AI refers to systems designed to use tools and make decisions to achieve a goal. \n    Unlike standard chatbots, AI agents can use 'Reasoning and Acting' (ReAct) patterns to \n    call APIs, search the web, or execute code independently to complete multi-step tasks.\n    ",
 '\n    Cloud-Native Microservices and Scalability\n\n    Modern backend architectures often utilize Spring Boot and Docker to create microservices. \n    Deploying these on AWS using services like EKS or Lambda allows for elastic scaling. \n    API Gateways act as the entry point, r

In [10]:
#Save sample
import tempfile
temp_dir = tempfile.mkdtemp()


for i , doc in enumerate(sample_docs):
    with open(f"doc.{i}.txt", "w") as f:
        f.write(doc)


In [16]:
#Document Loading
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    "Data",
    glob ="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={'encoding':'utf-8'}
)

In [17]:
loader

In [18]:
documents = loader.load()

In [19]:
documents

[Document(metadata={'source': 'Data\\doc.0.txt'}, page_content="\n    Retrieval-Augmented Generation (RAG) and Vector Databases\n\n    RAG systems enhance Large Language Models by retrieving relevant data from external sources like ChromaDB or FAISS. \n    By converting documents into vector embeddings, the system can perform semantic searches to find context \n    that a model wasn't originally trained on, reducing hallucinations.\n    "),
 Document(metadata={'source': 'Data\\doc.1.txt'}, page_content="\n    Agentic AI and Autonomous Workflows\n\n    Agentic AI refers to systems designed to use tools and make decisions to achieve a goal. \n    Unlike standard chatbots, AI agents can use 'Reasoning and Acting' (ReAct) patterns to \n    call APIs, search the web, or execute code independently to complete multi-step tasks.\n    "),
 Document(metadata={'source': 'Data\\doc.2.txt'}, page_content='\n    Cloud-Native Microservices and Scalability\n\n    Modern backend architectures often uti

In [22]:
#Text Splliting 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0 , length_function = len , separators=[" "] )

chunk = text_splitter.split_documents(documents)

In [23]:
chunk

[Document(metadata={'source': 'Data\\doc.0.txt'}, page_content='Retrieval-Augmented Generation (RAG) and Vector Databases\n\n    RAG systems enhance Large'),
 Document(metadata={'source': 'Data\\doc.0.txt'}, page_content='Language Models by retrieving relevant data from external sources like ChromaDB or FAISS. \n    By'),
 Document(metadata={'source': 'Data\\doc.0.txt'}, page_content='converting documents into vector embeddings, the system can perform semantic searches to find'),
 Document(metadata={'source': 'Data\\doc.0.txt'}, page_content="context \n    that a model wasn't originally trained on, reducing hallucinations."),
 Document(metadata={'source': 'Data\\doc.1.txt'}, page_content='Agentic AI and Autonomous Workflows\n\n    Agentic AI refers to systems designed to use tools and'),
 Document(metadata={'source': 'Data\\doc.1.txt'}, page_content="make decisions to achieve a goal. \n    Unlike standard chatbots, AI agents can use 'Reasoning and"),
 Document(metadata={'source': 'Data

In [24]:
#Embedding Models 
embeddings = OpenAIEmbeddings()


In [27]:
#Chroma DB Vector Store 
persistence_directory = "./chrom_db"

vectorStore = Chroma.from_documents(
    documents=chunk,
    embedding=OpenAIEmbeddings(),
    persist_directory=persistence_directory,
    collection_name="rag_collection"
)

In [32]:
#Text Similarity Search 
query = "What refers to Agentic AI?"

similar_docs = vectorStore.similarity_search(query , k=3)
similar_docs

[Document(id='fed22f43-38e9-4be4-8e35-d2ec3e2d5f9d', metadata={'source': 'Data\\doc.1.txt'}, page_content='Agentic AI and Autonomous Workflows\n\n    Agentic AI refers to systems designed to use tools and'),
 Document(id='b7ed2ebe-41fc-444e-88e4-c2fc37b0dcab', metadata={'source': 'Data\\doc.3.txt'}, page_content='Natural Language Processing with NLTK and Transformers\n\n    Natural Language Processing (NLP)'),
 Document(id='0a3feb73-abe5-4cf1-bbb1-bb50110b138f', metadata={'source': 'Data\\doc.1.txt'}, page_content="make decisions to achieve a goal. \n    Unlike standard chatbots, AI agents can use 'Reasoning and")]

In [33]:
#Advacne similarity search 
score = vectorStore.similarity_search_with_score(query , k =3)
score

[(Document(id='fed22f43-38e9-4be4-8e35-d2ec3e2d5f9d', metadata={'source': 'Data\\doc.1.txt'}, page_content='Agentic AI and Autonomous Workflows\n\n    Agentic AI refers to systems designed to use tools and'),
  0.2739553451538086),
 (Document(id='b7ed2ebe-41fc-444e-88e4-c2fc37b0dcab', metadata={'source': 'Data\\doc.3.txt'}, page_content='Natural Language Processing with NLTK and Transformers\n\n    Natural Language Processing (NLP)'),
  0.4770853817462921),
 (Document(id='0a3feb73-abe5-4cf1-bbb1-bb50110b138f', metadata={'source': 'Data\\doc.1.txt'}, page_content="make decisions to achieve a goal. \n    Unlike standard chatbots, AI agents can use 'Reasoning and"),
  0.4980132281780243)]

In [34]:
from langchain_groq import ChatGroq

In [36]:
llm = ChatGroq(
    model= "qwen/qwen3-32b",
    temperature= 0.2,
    max_tokens=500
)

In [39]:
llm.invoke("What is Large Language Model")

AIMessage(content="<think>\nOkay, the user is asking what a large language model is. First, I need to define it clearly. A large language model is a type of artificial intelligence that processes and generates human-like text. It's based on deep learning techniques, especially neural networks.\n\nNext, I should explain the key components. They are trained on vast amounts of text data, which allows them to understand grammar, context, and even some reasoning. The training process involves predicting the next word in a sentence, which helps them learn patterns and relationships between words.\n\nThen, I should mention the different architectures, like the Transformer model, which is widely used. Transformers use attention mechanisms to handle long-range dependencies in text, making them more efficient and effective than older models like RNNs.\n\nApplications are important to highlight. They can be used for tasks like translation, summarization, question-answering, and even creative writ

In [40]:
llm = init_chat_model("groq:qwen/qwen3-32b")


In [41]:
#Rag Chain
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [43]:
#Convert vector store ---> Retriever
retriever = vectorStore.as_retriever(
    search_kwargs={"k":3}

)
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000211C76BFD90>, search_kwargs={'k': 3})

In [44]:
#Create a Prompt Template
system_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
If you don't know the answer just say don't know.
Use three sentence minimum and keep the answer concise.

Context: {context}

"""



In [45]:
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder

In [46]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [47]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer just say don't know.\nUse three sentence minimum and keep the answer concise.\n\nContext: {context}\n\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [48]:
document_chain = create_stuff_documents_chain(llm,prompt)

In [49]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer just say don't know.\nUse three sentence minimum and keep the answer concise.\n\nContext: {context}\n\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False

In [50]:
rag_chain = create_retrieval_chain(retriever , document_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000211C76BFD90>, search_kwargs={'k': 3}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question.\nIf you don't kno

In [53]:
response = rag_chain.invoke({"input": "What is agentic AI?"})

In [54]:
response['answer']

'<think>\nOkay, the user is asking, "What is agentic AI?" Let me start by recalling the context provided. The context mentions that agentic AI refers to systems designed to use tools and handle complex tasks like sentiment analysis and language translation. Also, there\'s a mention of Natural Language Processing with NLTK and Transformers.\n\nFirst, I need to define agentic AI clearly. The key points from the context are that it\'s about systems using tools for complex tasks. But I should expand a bit to make sure the answer is comprehensive. Maybe explain what "agentic" means in this context—like autonomy, decision-making, using NLP techniques.\n\nWait, the context also connects it to NLP tasks. So, I should mention examples like sentiment analysis and translation. I should link agentic AI to NLP technologies like NLTK and Transformers frameworks, as the context does. That shows how these systems are implemented.\n\nThe user might be looking to understand the practical applications or

In [55]:
#For querying a set of questions
def query_rag_modern(question):
    print(f"Question:{question}")
    print("-"*50)

    result = rag_chain.invoke({"input":question})

    print(f"Answer: {result['answer']}")
    
    return result

text_question = [
    "What does agentic AI refers to ?"
    "What is said in cloud native in the document?"
    "what is RAG?"
]

for question in text_question:
    result = query_rag_modern(question)
    print("\n"+ "="*80+ "\n")
    

Question:What does agentic AI refers to ?What is said in cloud native in the document?what is RAG?
--------------------------------------------------
Answer: <think>
Okay, let's break down the user's questions. They asked three things: what agentic AI refers to, what is said about cloud-native in the document, and what RAG is.

First, looking at the context provided. The first section is about Agentic AI and Autonomous Workflows. The user's first question is directly about that. The context says Agentic AI refers to systems designed to use tools and... Hmm, the context here seems cut off. But from what's given, I can mention that agentic AI systems use tools and autonomous workflows to perform tasks, possibly with RAG and vector databases.

Next, the user is asking about cloud-native in the document. The third section is titled "Cloud-Native Microservices and Scalability" and mentions modern backend architectures using cloud-native. The context here is also cut off, but I can infer tha

In [56]:
#Parser 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough , RunnableParallel


In [57]:
custom_prompt = ChatPromptTemplate.from_template(
    """Use the following context to answer the question. IF you don't know the answer say you don't know.
    Provide Specific details from the context to support your answer.
    
    Context:{context}
    Question:{question}
    Answer: 

    """
)

In [58]:
custom_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following context to answer the question. IF you don't know the answer say you don't know.\n    Provide Specific details from the context to support your answer.\n\n    Context:{context}\n    Question:{question}\n    Answer: \n\n    "), additional_kwargs={})])

In [67]:
#Build the chain using LCEL
rag_chain_lcel = (
    {"context": retriever,
     "question": RunnablePassthrough()}
     |custom_prompt
     |llm
     |StrOutputParser()
)

In [68]:
rag_chain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000211C76BFD90>, search_kwargs={'k': 3}),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following context to answer the question. IF you don't know the answer say you don't know.\n    Provide Specific details from the context to support your answer.\n\n    Context:{context}\n    Question:{question}\n    Answer: \n\n    "), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, clie

In [69]:
rag_chain_lcel.invoke("What is Agentic AI?")

'<think>\nOkay, let\'s see. The user is asking about Agentic AI. I need to use the provided context to answer this. Let me check the documents.\n\nThe first document has ID \'fed22f43-38e9-4be4-8e35-d2ec3e2d5f9d\' and mentions "Agentic AI refers to systems designed to use tools and". That seems relevant. The rest of the context includes other documents about NLP, which might not be related. \n\nThe user wants a specific answer with details from the context. The first document\'s page content starts with "Agentic AI refers to systems designed to use tools and" but it\'s cut off. Since there\'s no more information in the other documents, I have to use that partial sentence. I should mention that the context defines Agentic AI as systems designed to use tools, even though the definition is incomplete. I can\'t add anything else because the other documents don\'t talk about Agentic AI. Also, the user said to say "don\'t know" if the answer isn\'t there, but since there\'s a partial answer,

In [70]:
#Adding New Documents in the vector Store
new_document = """
Reinforcement Learning in Detail

Reinforcement learning (RL) is a type of machine learning where an agent learns to make
decisions by interacting with an environment. The agent receives rewards or penalties
based on its actions and learns to maximize cumulative reward over time. Key concepts
in RL include: states, actions, rewards, policies, and value functions. Popular RL
algorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and
Actor-Critic methods. RL has been successfully applied to game playing (like AlphaGo),
robotics, and autonomous systems.
"""

In [71]:
new_doc = Document(
    page_content=new_document,
    metadata={"source": "manual_addition" , "topic": "reinforcement_learning"}
)

In [72]:
new_doc

Document(metadata={'source': 'manual_addition', 'topic': 'reinforcement_learning'}, page_content='\nReinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where an agent learns to make\ndecisions by interacting with an environment. The agent receives rewards or penalties\nbased on its actions and learns to maximize cumulative reward over time. Key concepts\nin RL include: states, actions, rewards, policies, and value functions. Popular RL\nalgorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and\nActor-Critic methods. RL has been successfully applied to game playing (like AlphaGo),\nrobotics, and autonomous systems.\n')

In [73]:
new_chunk = text_splitter.split_documents([new_doc])

In [74]:
new_chunk

[Document(metadata={'source': 'manual_addition', 'topic': 'reinforcement_learning'}, page_content='Reinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where'),
 Document(metadata={'source': 'manual_addition', 'topic': 'reinforcement_learning'}, page_content='an agent learns to make\ndecisions by interacting with an environment. The agent receives rewards or'),
 Document(metadata={'source': 'manual_addition', 'topic': 'reinforcement_learning'}, page_content='penalties\nbased on its actions and learns to maximize cumulative reward over time. Key concepts\nin'),
 Document(metadata={'source': 'manual_addition', 'topic': 'reinforcement_learning'}, page_content='RL include: states, actions, rewards, policies, and value functions. Popular RL\nalgorithms include'),
 Document(metadata={'source': 'manual_addition', 'topic': 'reinforcement_learning'}, page_content='Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and\nActor-Critic methods. RL h

In [76]:
vectorStore.add_documents(new_chunk)

['f0fd363f-7bc7-40e1-9791-a05c3d02d093',
 '994dcd22-560d-4566-a599-972f3b97ef5f',
 'fdf01580-8ec4-47fd-8cd9-b00bc5bb90ee',
 '52effe70-0d2d-41bd-8d23-5ceb5a97d687',
 '14193c68-b48b-4def-a70a-7e381868844e',
 '14447d5c-5782-4185-aaf9-9aeb09512d1c']

In [77]:
new_question = "What is reinforcement learning?"
result = query_rag_modern(new_question)
result

Question:What is reinforcement learning?
--------------------------------------------------
Answer: <think>
Okay, the user is asking, "What is reinforcement learning?" I need to answer this using the provided context.

Looking at the context, it starts by stating that reinforcement learning (RL) is a type of machine learning. The first sentence is repeated, which might be a formatting error, but the key points are there. Then it mentions some algorithms like Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and Actor-Critic methods. It also says RL has been... but the context cuts off there.

So, the answer should define RL as a machine learning type where an agent learns by interacting with an environment to maximize cumulative reward. The main components are the agent, environment, actions, states, and rewards. The mentioned algorithms are examples of RL methods. Since the context doesn't provide more details, I should stick to the given info and not add anything beyond tha

{'input': 'What is reinforcement learning?',
 'context': [Document(id='3d1fbd3d-50ce-44be-8499-9441847dac91', metadata={'topic': 'reinforcement_learning', 'source': 'manual_addition'}, page_content='Reinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where'),
  Document(id='f0fd363f-7bc7-40e1-9791-a05c3d02d093', metadata={'source': 'manual_addition', 'topic': 'reinforcement_learning'}, page_content='Reinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where'),
  Document(id='14193c68-b48b-4def-a70a-7e381868844e', metadata={'topic': 'reinforcement_learning', 'source': 'manual_addition'}, page_content='Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and\nActor-Critic methods. RL has been')],
 'answer': '<think>\nOkay, the user is asking, "What is reinforcement learning?" I need to answer this using the provided context.\n\nLooking at the context, it starts by stating that reinforcement learning

In [78]:
#Conversational Memory

In [79]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage , AIMessage

In [80]:
#create a prompt that includes the chat history
contextualize_q_system_prompt = """Given a chat history and the latest user question 
which might reference context in chat history, formulate a standalone question
which can be understood without chat history. DO NOT answer the question,
just reformulate it if needed and otherwise return it as it is."""

In [81]:
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

In [82]:
#create history 
history_aware_retriever = create_history_aware_retriever(llm , retriever ,  contextualize_q_prompt)

In [83]:
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000211C76BFD90>, search_kwargs={'k': 3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessa

In [84]:
# Create a new document chain with history
qa_system_prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Context: {context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever,
    question_answer_chain
)

In [85]:
chat_hsitory = []

result1 = conversational_rag_chain.invoke({
    "chat_history": chat_hsitory,
    "input": "What is reinforcement learning?"
})


In [86]:
print(result1['answer'])

<think>
Okay, the user is asking, "What is reinforcement learning?" Let me check the context provided.

The context starts by stating that reinforcement learning (RL) is a type of machine learning. Then it mentions Q-learning, DQN, Policy Gradient, and Actor-Critic methods as part of RL. It also says RL has been... but the context cuts off there.

So, the key points from the context are: RL is a machine learning type where an agent learns by interacting with an environment. The answer should include that it's about learning optimal actions through trial and error, using rewards and penalties. The examples like Q-learning and DQN are important to mention as methods within RL.

Wait, the user wants a concise answer in three sentences max. Let me structure that. First sentence defines RL. Second explains how it works (agent, environment, rewards). Third lists some methods. Need to make sure not to include any extra info beyond what's in the context. Also, avoid any markdown and keep it na

In [87]:
chat_hsitory.extend([
    HumanMessage(content="What is reinforcement learning?"),
    AIMessage(content=result1['answer'])
])

In [89]:
result2 = conversational_rag_chain.invoke({
    "chat_history" : chat_hsitory,
    "input": "What are its types?"
})

In [90]:
result2['answer']

'<think>\nOkay, the user is asking about the types of reinforcement learning. Let me check the previous conversation. In the first question, they asked what reinforcement learning is, and I mentioned Q-learning, DQN, Policy Gradient methods, and Actor-Critic methods.\n\nNow, the user is following up with "What are its types?" So they want more details on the different types. The context provided includes Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and Actor-Critic methods. \n\nI need to list these types clearly. But wait, are there more types beyond what\'s in the context? The context is limited, so I should stick to what\'s provided. The user might not need an exhaustive list but the main ones mentioned. \n\nAlso, the user might be looking for a brief explanation of each type. However, the context doesn\'t provide detailed explanations, just the names. Since the instruction says to use the retrieved context and not make up information, I should list the types mentioned